In [ ]:
!pip install emoji
!pip install scikit-learn==1.0.1

     |████████████████████████████████| 170 kB 29.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=2d352e3344227939d7d7216fd2db38efb14731320f023806827b54d0c2f379c7
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


# Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import string, re, requests, csv
from google.colab import drive
from wordcloud import WordCloud
from gensim.corpora import WikiCorpus

In [ ]:
from nltk import word_tokenize, sent_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load dataset

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

train.head()

,text,label,username,likes,datetime
0,🔥,negative,lanyardsemarang,0,2020-11-02
1,Naik Turun maju can5ik 😂😂😂😂,neutral,hadi_soeparno,1,2021-03-29
2,Prosentase tingkat kematian karena covid 19 ut...,neutral,heryadisaputro,0,2020-09-22
3,Rapid test/ test swab PCR Di DKK/dinkes Semara...,negative,airlangga15,0,2020-05-29
4,"Min, area pedurungan bs dilakukan dmn,",neutral,fauzanabell,0,2021-06-22


# EDA

## Wordcloud

In [ ]:
# positive comments before preprocessing
# data_pos = train[train['label'] == 'positive']

# all_text = ' '.join(word for word in data_pos['text'])
# wordcloud = WordCloud(colormap='Greens', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text)
# plt.figure(figsize=(20,10))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.margins(x=0, y=0)
# plt.show()

In [ ]:
# negative comments before preprocessing
# data_neg = train[train['label'] == 'negative']

# all_text = ' '.join(word for word in data_neg['text'])
# wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text)
# plt.figure(figsize=(20,10))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.margins(x=0, y=0)
# plt.show()

In [ ]:
# neutral comments before preprocessing
# data_neut = train[train['label'] == 'neutral']

# all_text = ' '.join(word for word in data_neut['text'])
# wordcloud = WordCloud(colormap='Blues', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text)
# plt.figure(figsize=(20,10))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.margins(x=0, y=0)
# plt.show()

In [ ]:
# value counts
train['label'].value_counts()

neutral     2926
negative    2775
positive    2266
Name: label, dtype: int64

# Preprocess

In [ ]:
train_text = train['text']
test_text = test['text']

## Rename username

In [ ]:
pattern = "(?:@)([A-Za-z0-9_](?:(?:[A-Za-z0-9_]|(?:\.(?!\.))){0,28}(?:[A-Za-z0-9_]))?)"

train_text = train_text.apply(lambda x: re.sub(pattern, "@username", x))
test_text = test_text.apply(lambda x: re.sub(pattern, "@username", x))

## Cleansing

In [ ]:
def cleansing(data):

    # lowercasing
    data = data.lower()

    # remove punctuation
    punct = string.punctuation
    translator = str.maketrans(punct, ' '*len(punct))
    data = data.translate(translator)

    # remove ASCII dan unicode
    # data = data.encode('ascii', 'ignore').decode('utf-8')
    # data = re.sub(r'[^\x00-\x7f]',r'', data)
    
    # remove newline
    data = data.replace('\n', ' ')

    # remove digit
    pattern = r'[0-9]'
    data = re.sub(pattern, '', data)

    # remove extra space
    data = ' '.join(data.split())
    
    return data

In [ ]:
train_text = train_text.apply(lambda x: cleansing(x))
test_text = test_text.apply(lambda x: cleansing(x))

In [ ]:
train_text.head()

0                                                    🔥
1                           naik turun maju canik 😂😂😂😂
2    prosentase tingkat kematian karena covid utk w...
3    rapid test test swab pcr di dkk dinkes semaran...
4                 min area pedurungan bs dilakukan dmn
Name: text, dtype: object

## Remove emojis

In [ ]:
# import sys

# def remove_emoji(data):
#     emoji_pattern = re.compile("["
#                            u"\U0001F600-\U0001F64F"  # emoticons
#                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            u"\U00002702-\U000027B0"
#                            u"\U000024C2-\U0001F251"
#                            "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r' ', data)

In [ ]:
# train_text = train_text.apply(lambda x: remove_emoji(x))
# test_text = test_text.apply(lambda x: remove_emoji(x))

In [ ]:
# train_text.head()

# Replace emoji with text

In [ ]:
df_emoji = pd.read_csv('emoji_to_text.csv')
df_emoji.head()

,emoji,makna
0,😴,Wajah Tertidur
1,🤤,Meneteskan Air Liur
2,😷,Wajah Ditutupi Masker
3,🤒,Wajah dengan Termometer
4,🤕,Wajah Diperban


In [ ]:
UNICODE_EMO = {row['emoji']:row['makna'] for idx,row in df_emoji.iterrows()}

In [ ]:
import emoji
import functools
import operator
import re

def convert_emojis(text):
    # split emojis
    em_split_emoji = emoji.get_emoji_regexp().split(text)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    text = ' '.join(em_split)

    # convert emojis
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text.lower()

In [ ]:
train_text = train_text.apply(lambda x: convert_emojis(x))
test_text = test_text.apply(lambda x: convert_emojis(x))

In [ ]:
train.iloc[1], train_text.iloc[1]

(text        Naik Turun maju can5ik 😂😂😂😂
 label                           neutral
 username                  hadi_soeparno
 likes                                 1
 datetime                     2021-03-29
 Name: 1, dtype: object,
 'naik turun maju canik wajah_tertawa_sampai_menangis wajah_tertawa_sampai_menangis wajah_tertawa_sampai_menangis wajah_tertawa_sampai_menangis')

## Normalize kata alay/colloquial

In [ ]:
# CONSTRUCT KAMUS ALAY
text_path1 = 'https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/kbba.txt'
text_path2 = 'https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv'
kamus_alay1 = pd.read_csv(text_path1, delimiter="\t", header=None, names=['slang', 'formal'])
kamus_alay2 = pd.read_csv(text_path2)
kamus_alay = pd.concat([kamus_alay1, kamus_alay2[['slang', 'formal']]]).reset_index(drop=True)

dict_alay = dict()
for index, row in kamus_alay.iterrows():
    dict_alay[row['slang']] = row['formal']

def normalize_text(data):
  word_tokens = word_tokenize(data)
  result = [dict_alay.get(w,w) for w in word_tokens]
  return ' '.join(result)

In [ ]:
train_text = train_text.apply(lambda x: normalize_text(x))
test_text = test_text.apply(lambda x: normalize_text(x))

In [ ]:
train_text.head()

0                                                  api
1    naik turun maju canik wajah_tertawa_sampai_men...
2    prosentase tingkat kematian karena covid untuk...
3    rapid test test swab pcr di dan kawan-kawan di...
4            min area pedurungan bisa dilakukan dimana
Name: text, dtype: object

## Spelling correction (not implemented yet)

## Remove stopwords

In [ ]:
# CONSTRUCT STOPWORDS
rama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
yutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"
fpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"
sastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"
aliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"
pebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"
elang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"
nltk_stopword = stopwords.words('indonesian')

path_stopwords = [rama_stopword, yutomo_stopword, fpmipa_stopword, sastrawi_stopword, 
                  aliakbar_stopword, pebahasa_stopword, elang_stopword]

# CUSTOM STOPWORDS
other = '''
admin mimin min minkes kalo nya username
'''

# gabungkan stopwords
stopwords_l = nltk_stopword
for path in path_stopwords:
    response = requests.get(path)
    stopwords_l += response.text.split('\n')

st_words = set(stopwords_l)
other_stopword = set(other.split())

stop_words = st_words | other_stopword


def remove_stopword(text, stop_words=stop_words):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

In [ ]:
train_text = train_text.apply(lambda x: remove_stopword(x))
test_text = test_text.apply(lambda x: remove_stopword(x))

In [ ]:
train_text.head()

0                                                  api
1    turun maju canik wajah_tertawa_sampai_menangis...
2        prosentase kematian covid warga kota semarang
3    rapid test test swab pcr kawan-kawan dinkes se...
4                                      area pedurungan
Name: text, dtype: object

# Feature extraction (TF-IDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=2, max_df=0.95, max_features = 5000, ngram_range = (1, 3),
                             sublinear_tf = True )

train_features = vectorizer.fit_transform(train_text)
test_features = vectorizer.transform(test_text)

In [ ]:
train_features.toarray().shape

(7967, 5000)

In [ ]:
# export tf-idf vectorizer
# from joblib import dump, load

# dump(vectorizer, 'tfidf-vectorizer.joblib')

In [ ]:
# vct = load('tfidf-vectorizer.joblib')
# vct.transform(test_text).shape

# Random Forest

In [ ]:
# mapping label
mapper = {'neutral':0, 'positive':1, 'negative':2}
train_y = train['label'].map(mapper)
test_y = test['label'].map(mapper)

# train_y

In [ ]:
%%time
# from sklearn.svm import SVC

# clf = SVC()
# clf.fit(train_features, train_y)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_features, train_y)

CPU times: user 8.7 s, sys: 32 ms, total: 8.74 s
Wall time: 8.7 s


In [ ]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
%%time
# cross-val score
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, train_features, train_y, cv=5)
print(scores)

[0.71894605 0.72710163 0.72881356 0.73195229 0.7131199 ]
CPU times: user 36.2 s, sys: 78.9 ms, total: 36.3 s
Wall time: 36.1 s


In [ ]:
from joblib import dump, load

dump(clf, 'tfidf_rf.joblib') 

['tfidf_rf.joblib']

In [ ]:
# predict
y_pred = clf.predict(test_features)

In [ ]:
%%time
# accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

0.7228915662650602
CPU times: user 920 µs, sys: 0 ns, total: 920 µs
Wall time: 931 µs


In [ ]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# from joblib import dump, load

# dump(clf, 'tfidf_svc.joblib') 

GridSearchCV

In [ ]:
# train 1
## kernel = linear
## C = 1
## gamma = scale
## dfs = ovo

# train 2
## kernel = rbf
## C = 1
## gamma = scale
## dfs = ovo

In [ ]:
%%time
# Randomized Search
# references: 
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

clf = RandomForestClassifier()
clf_random = RandomizedSearchCV(estimator=clf, 
                                param_distributions=random_grid, 
                                n_iter=100, 
                                cv=5, 
                                verbose=2, random_state=42, n_jobs=-1)
# Fit the random search model
clf_random.fit(train_features, train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 1min 36s, sys: 4.44 s, total: 1min 41s
Wall time: 1h 46min 32s


In [ ]:
clf_random.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 1600}

In [ ]:
y_pred = clf_random.predict(test_features)

accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

0.7274096385542169


In [ ]:
clf_random.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator': RandomForestClassifier(),
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'n_iter': 100,
 'n_jobs': -1,
 'param_distributions': {'bootstrap': [True, False],
  'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
  'max_features': ['auto', 'sqrt'],
  'min_samples_leaf': [1, 2, 4],
  'min_samples_split': [2, 5, 10],
  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1

In [ ]:
clf_random.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 1600}

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# params = {'kernel': ['linear', 'rbf', 'poly'],  # 3
#           'C': [1, 0.25, 0.5, 0.75],            # 4
#           'gamma': ['scale', 'auto', 1, 2, 3],  # 5
#           'decision_function_shape': ['ovo','ovr'] # 2
#           # 3 * 4 * 5 *2 * 5cv = 120 * 5 = 600 kombinasi
# }

# param_grid = { 
#     'n_estimators': [100, 200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [2,3,4,5,6],
#     'min_samples_leaf': [1,2,3],
#     'criterion' :['gini', 'entropy']
# }

param_grid = {
    'bootstrap': [True],
    # 'max_depth': [100, 110, None],
    # 'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [8, 10, 12],
    # 'criterion' :['gini'],
    'n_estimators': [800, 1200, 1500, 1800]
}

clf = RandomForestClassifier()
# cv_test = StratifiedKFold(n_splits=5, shuffle=True)
clf_grid = GridSearchCV(clf, param_grid, cv=5)
clf_grid.fit(train_features, train_y)

CPU times: user 1h 37min 31s, sys: 12.9 s, total: 1h 37min 44s
Wall time: 1h 37min 27s


In [ ]:
# accuracy test
y_pred = clf_grid.predict(test_features)

accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

0.7289156626506024


In [ ]:
clf_grid.best_params_

{'bootstrap': True,
 'min_samples_leaf': 1,
 'min_samples_split': 12,
 'n_estimators': 1800}

In [ ]:
%%time
# cross-val score
scores = cross_val_score(clf_grid, train_features, train_y, cv=5)
print(scores)

[0.71706399 0.73525721 0.73320778 0.73822976 0.72441933]
CPU times: user 9h 31min 4s, sys: 1min 8s, total: 9h 32min 12s
Wall time: 9h 30min 14s


In [ ]:
# param_grid = { 
#     'n_estimators': [100, 200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [2,3,4,5,6],
#     'min_samples_leaf': [1,2,3],
#     'criterion' :['gini', 'entropy']
# }

# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [80, 90, 100, 110],
#     'max_features': [2, 3],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [100, 200, 300, 1000]
# }

# clf = RandomForestClassifier()
# clf_grid = GridSearchCV(clf, param_grid)
# clf_grid.fit(train_features, train_y)

# print(clf.best_params_)

# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(train_features, train_y, 
#                                                  test_size=0.33, 
#                                                  random_state=2021)
# clf_grid = GridSearchCV(clf, param_grid)
# clf_grid.fit(X_train, y_train)

# print(clf.best_params_)

In [ ]:
# accuracy test
y_pred = clf_grid.predict(test_features)

accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

0.7289156626506024


In [ ]:
from joblib import dump, load

dump(clf_grid, 'tfidf_rf_tuned.joblib') 

['tfidf_rf_tuned.joblib']

In [ ]:
# a = load("svc_tuned.joblib")
# b = a.predict(test_features)
# accuracy_score(test_y, b)